# Using tool calling to do math
One great use of tool calling is to perform math operations, which LLMs are not typically good at doing on their own.

To get started, first install dependencies.

In [ ]:
%pip install writer-sdk -q

### Initialization

The cell below performs the initialization required for this notebook including the creation of an instance of the `Writer` object to interact with the LLM.

To create a Writer client object, you need an API key. [You can sign up for one for free](https://app.writer.com/register). 

Once you have an API key, we recommend that you store it as an environment variable in a `.env` file like so:

```
WRITER_API_KEY="{Your Writer API key goes here}"
```

When you instantiate the client with `client = Writer()`, the newly-created object will automatically look for an environment variable named `WRITER_API_KEY` and will complete the instantiation if an only if `WRITER_API_KEY` has been defined. This notebook uses the [python-dotenv] library to automatically define environment variables based on the contents of an `.env` file in the same directory.

The `Writer()` initializer method also has an `api_key` parameter that you can use like this...

```
client = Writer(api_key="{Your Writer API key goes here}")
```

...but we strongly encourage you not to leave API keys in your source code.

In [ ]:
from writerai import Writer
import math

# Load environment variables from .env file
%reload_ext dotenv
%dotenv

client = Writer()

### Define your math functions

Next, define your math functions. Here we are defining a function to calculate the square root and a function to calculate factorials.

In [ ]:
# Define the math functions
def calculate_square_root(number):
    return math.sqrt(number)

def calculate_factorial(number):
    return math.factorial(number)

### Create a `tools` array
Next, create a JSON schema for each function inside of a `tools` array. Each function should have:

- A `type` of `function`
- A `name`, a `description`
- A `parameters` object
  - In the case of these two functions, the `parameters` object should have a `type` of `object`, a `properties` object with a single parameter called `number` defined as an `integer`, and a `required` array with the name of the parameter.


In [ ]:
# Define the tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_square_root",
            "description": "Calculate the square root of a number",
            "parameters": {
                "type": "object",
                "properties": {
                    "number": {"type": "number", "description": "The number to calculate the square root of"}
                },
                "required": ["number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_factorial",
            "description": "Calculate the factorial of a number",
            "parameters": {
                "type": "object",
                "properties": {
                    "number": {"type": "integer", "description": "The number to calculate the factorial of"}
                },
                "required": ["number"]
            }
        }
    }
]

### Send the `tools` array with your messages
Pass the `tools` array along with the `messages` array to the `chat` method of the SDK. The `tool_choice` parameter is set to `auto`, which means that the model will decide which of the tools you've defined to use, if any. Feel free to edit the contents of the `messages` array to a different math problem.

After you have a response, you'll append the response message (accessed via `response.choices[0].message`) to the `messages` array.

In [ ]:
# Set up the conversation
messages = [{"role": "user", "content": "What's the square root of 16 and the factorial of 5?"}]

# Send the messages and tools to the model
response = client.chat.chat(
    model="palmyra-x-004", messages=messages, tools=tools, tool_choice="auto", stream=False
)

# Append the response message to the messages array
response_message = response.choices[0].message
messages.append(response_message)

print(f"Tool call response message object:\n{response_message}\n")

### Process the response and handle tool calls
When the model decides to use a tool, it will add a `tool_calls` array to the response message. Loop through the `tool_calls` array of the response message and call the appropriate function. 

Then, append the result of the function to the `messages` array. This message should have the following structure:

- `role`: `"tool"`
- `tool_call_id`: id of the tool call, accessed via `tool_call.id`
- `name`: name of the function that was called, accessed via `tool_call.function.name`
- `content`: the result of the function

In [ ]:
# Process the response and handle tool calls
tool_calls = response_message.tool_calls
for tool_call in tool_calls:
    if tool_call.function.name == "calculate_square_root":
        number = eval(tool_call.function.arguments)["number"]
        function_response = calculate_square_root(number)
    elif tool_call.function.name == "calculate_factorial":
        number = eval(tool_call.function.arguments)["number"]
        function_response = calculate_factorial(number)
    
    messages.append(
        {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(function_response),
        }
    )

### Get the final response
Finally, get the final response from the model. It should be the answer to the math problem(s) you asked the model.

In [ ]:
final_response = client.chat.chat(
        model="palmyra-x-004", messages=messages, stream=False
    )

print(f"Final response: \n{final_response.choices[0].message.content}\n")

That's it! You've successfully used tool calling to perform math operations. To learn more about tool calling, check out the [tool calling guide](https://dev.writer.com/api-guides/tool-calling) on the Writer docs.